Import libraries for data cleaning 

In [16]:
import csv
from tokenize import String

from notebooks.db_connection import get_db_engine
from notebooks.db_connection import execute_query
import numpy as np
import pandas as pd
import seaborn as sns
import os
import re

Reading CSV file

In [17]:
df_studios = pd.read_csv(r"C:\Users\ricca\Desktop\CSV\studios.csv\studios.csv")

Top records of CSV file: studios.csv

In [18]:
df_studios

,id,studio
0,1000001,LuckyChap Entertainment
1,1000001,Heyday Films
2,1000001,NB/GG Pictures
3,1000001,Mattel
4,1000001,Warner Bros. Pictures
...,...,...
679278,1941596,上海猫眼影业有限公司
679279,1941596,坏小子（北京）传媒有限公司
679280,1941596,亚太国影（重庆）文化传媒有限公司
679281,1941596,凤凰传奇影业有限公司


Checking for total dimension of the csv

In [19]:
df_studios.shape

(679283, 2)

Rename 'id' column in 'id_studio'. "inplace = True" modifies directly the df without creating a copy

In [20]:
df_studios.rename(columns = {"id":"id_movie"} , inplace = True)


In [21]:
df_studios.columns

Index(['id_movie', 'studio'], dtype='object')

Check info about column type

In [22]:
df_studios.dtypes

id_movie     int64
studio      object
dtype: object

Correction of data type of the records

In [23]:
df_studios['id_movie'] = df_studios['id_movie'].astype('Int64')
df_studios['studio'] = df_studios['studio'].fillna(pd.NA).astype('string')
df_studios.dtypes

id_movie             Int64
studio      string[python]
dtype: object

Check if there are duplicates

In [24]:
df_studios.duplicated().sum()


np.int64(212)

In [25]:
df_studios.drop_duplicates(inplace = True)
df_studios.duplicated().sum()

np.int64(0)

Check the missing values

In [26]:
df_studios.isnull().sum()

id_movie     0
studio      10
dtype: int64

Dropping records with missing value 

In [27]:
df_studios = df_studios.dropna()
df_studios.isnull().sum()

id_movie    0
studio      0
dtype: int64

Remove studios where id_movie doesn't exist

In [28]:
valid_movies = pd.read_csv("movies_valid_ids.csv")
df_studios = df_studios[df_studios["id_movie"].isin(valid_movies["id_movie"])]
df_studios.shape

(679061, 2)

Connecting to PostgreSQL database and creating the table

In [29]:
engine = get_db_engine()
sql = \
    """CREATE TABLE IF NOT EXISTS studios (
            id BIGSERIAL PRIMARY KEY,
            id_movie INTEGER CHECK (id_movie>=0),
            studio TEXT CHECK (studio<>'')
            );
    """
execute_query(sql)

Query Executed Successfully!


Population of the table using .to_sql function


In [30]:
df_studios.to_sql("studios", engine, if_exists="append", index=False)

61